In [1]:
import polars as pl
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
sns.set_theme()

pl.Config.set_fmt_str_lengths(100)


polars.config.Config

In [2]:
file_path = "/Users/jeison/Desktop/OptionsTradingEnv/OptionsData/Data/SPY_prices.pkl"

In [3]:
with open(file_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
days = (
    data
    .group_by_dynamic("date", every="1d")
    .agg(
        pl.col('close').last(),
        pl.col('open').first()
    )
    .with_columns(
        pl.col("close")
        .reverse()
        .rolling_mean_by("date", window_size='5d', closed='right')
        .reverse()
        .alias("5-day Average")
    )
)
days.head(25)

date,close,open,5-day Average
datetime[ms],f64,f64,f64
2003-09-10 00:00:00,101.84,102.86,102.486
2003-09-11 00:00:00,102.31,101.96,102.774
2003-09-12 00:00:00,102.66,102.3,102.89
2003-09-15 00:00:00,102.02,102.63,102.966667
2003-09-16 00:00:00,103.6,102.25,103.836667
…,…,…,…
2003-10-08 00:00:00,103.91,104.21,104.644
2003-10-09 00:00:00,104.31,104.09,104.8275
2003-10-10 00:00:00,104.51,104.46,105.0


In [5]:
last_day = (
    data
    .group_by_dynamic("date", every='1d')
    .agg(
        pl.col('open').first(),
        pl.col('close').last()
    )
    .with_columns(pl.col('date').cast(pl.Datetime).dt.replace_time_zone("UTC"))
    .with_columns(pl.col('date').dt.convert_time_zone("America/New_York"))
    # .filter(pl.col('date').dt.date() == pl.lit(dt.datetime(2024, 5, 31)))
    # .filter(pl.col('date').dt.hour() < 16)
    # .filter(pl.col('date').dt.hour() >= 9)
    # .with_columns(pl.col('date').shift(-1).alias('date_shift'))
    # .filter(pl.col('date_shift').dt.month() != pl.col('date').dt.month())
    .filter(pl.col('date').dt.year() >= 2023)
)

with pl.Config(tbl_rows=100):
    print(last_day)

shape: (394, 3)
┌────────────────────────────────┬────────┬──────────┐
│ date                           ┆ open   ┆ close    │
│ ---                            ┆ ---    ┆ ---      │
│ datetime[μs, America/New_York] ┆ f64    ┆ f64      │
╞════════════════════════════════╪════════╪══════════╡
│ 2023-01-02 19:00:00 EST        ┆ 384.0  ┆ 380.79   │
│ 2023-01-03 19:00:00 EST        ┆ 380.79 ┆ 383.63   │
│ 2023-01-04 19:00:00 EST        ┆ 383.55 ┆ 380.0    │
│ 2023-01-05 19:00:00 EST        ┆ 380.0  ┆ 387.78   │
│ 2023-01-06 19:00:00 EST        ┆ 387.79 ┆ 387.58   │
│ 2023-01-08 19:00:00 EST        ┆ 388.37 ┆ 387.54   │
│ 2023-01-09 19:00:00 EST        ┆ 387.6  ┆ 390.7    │
│ 2023-01-10 19:00:00 EST        ┆ 390.75 ┆ 395.35   │
│ 2023-01-11 19:00:00 EST        ┆ 395.35 ┆ 397.11   │
│ 2023-01-12 19:00:00 EST        ┆ 397.13 ┆ 398.3    │
│ 2023-01-13 19:00:00 EST        ┆ 398.3  ┆ 398.4    │
│ 2023-01-16 19:00:00 EST        ┆ 397.95 ┆ 396.81   │
│ 2023-01-17 19:00:00 EST        ┆ 396.82 ┆ 391.5

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
with open("OptionsData/Data/SPY_options_tickers.pkl", 'rb') as file:
    tickers = pickle.load(file)
tickers

additional_underlyings,cfi,contract_type,correction,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker
null,str,str,null,str,str,str,i64,i64,str,str
null,"""OCASPS""","""put""",null,"""american""","""2022-01-03""","""BATO""",100,235,"""O:SPY220103P00235000""","""SPY"""
null,"""OCASPS""","""put""",null,"""american""","""2022-01-03""","""BATO""",100,240,"""O:SPY220103P00240000""","""SPY"""
null,"""OCASPS""","""put""",null,"""american""","""2022-01-03""","""BATO""",100,245,"""O:SPY220103P00245000""","""SPY"""
null,"""OCASPS""","""put""",null,"""american""","""2022-01-03""","""BATO""",100,250,"""O:SPY220103P00250000""","""SPY"""
null,"""OCASPS""","""put""",null,"""american""","""2022-01-03""","""BATO""",100,255,"""O:SPY220103P00255000""","""SPY"""
…,…,…,…,…,…,…,…,…,…,…
null,"""OCASPS""","""call""",null,"""american""","""2024-06-14""","""BATO""",100,590,"""O:SPY240614C00590000""","""SPY"""
null,"""OCASPS""","""call""",null,"""american""","""2024-06-14""","""BATO""",100,595,"""O:SPY240614C00595000""","""SPY"""
null,"""OCASPS""","""call""",null,"""american""","""2024-06-14""","""BATO""",100,600,"""O:SPY240614C00600000""","""SPY"""


In [8]:
with open("OptionsData/Data/SPY_options_history.pkl", 'rb') as file:
    history = pickle.load(file)
history

{'O:SPY220103P00360000': shape: (2, 10)
 ┌──────┬──────┬──────┬───────┬───┬───────────────┬──────────────┬──────┬─────────────────────┐
 │ open ┆ high ┆ low  ┆ close ┆ … ┆ timestamp     ┆ transactions ┆ otc  ┆ date                │
 │ ---  ┆ ---  ┆ ---  ┆ ---   ┆   ┆ ---           ┆ ---          ┆ ---  ┆ ---                 │
 │ f64  ┆ f64  ┆ f64  ┆ f64   ┆   ┆ i64           ┆ i64          ┆ null ┆ datetime[ms]        │
 ╞══════╪══════╪══════╪═══════╪═══╪═══════════════╪══════════════╪══════╪═════════════════════╡
 │ 0.01 ┆ 0.01 ┆ 0.01 ┆ 0.01  ┆ … ┆ 1641241380000 ┆ 2            ┆ null ┆ 2022-01-03 20:23:00 │
 │ 0.01 ┆ 0.01 ┆ 0.01 ┆ 0.01  ┆ … ┆ 1641241500000 ┆ 1            ┆ null ┆ 2022-01-03 20:25:00 │
 └──────┴──────┴──────┴───────┴───┴───────────────┴──────────────┴──────┴─────────────────────┘,
 'O:SPY220103P00370000': shape: (1, 10)
 ┌──────┬──────┬──────┬───────┬───┬───────────────┬──────────────┬──────┬─────────────────────┐
 │ open ┆ high ┆ low  ┆ close ┆ … ┆ timestamp     ┆ tra

In [9]:
import pandas as pd

In [10]:
pct_days = pd.read_csv("OptionsData/Data/pctdays.csv", header=0).iloc[:, :12]
pct_days.index = pct_days.index + 1
pct_days

,Tables January,Tables February,Tables March,Tables April,Tables May,Tables June,Tables July,Tables August,Tables September,Tables October,Tables November,Tables December
1,NaN,0.16%,0.30%,0.28%,0.19%,0.18%,0.26%,-0.05%,0.10%,0.23%,0.17%,0.06%
2,0.31%,0.16%,0.15%,0.08%,0.16%,0.16%,0.05%,0.08%,0.25%,0.16%,0.31%,0.08%
3,0.18%,0.14%,0.04%,0.00%,0.07%,0.01%,0.30%,0.08%,0.02%,-0.04%,0.39%,-0.15%
4,0.09%,-0.07%,0.20%,-0.05%,0.04%,-0.01%,NaN,-0.24%,-0.15%,0.17%,0.21%,0.14%
5,0.09%,-0.14%,0.07%,0.25%,0.14%,0.17%,0.14%,-0.14%,-0.07%,0.26%,0.29%,0.16%
6,0.13%,0.13%,-0.05%,0.18%,-0.14%,0.22%,0.09%,0.01%,0.11%,0.15%,-0.08%,0.16%
7,-0.07%,0.03%,-0.25%,-0.19%,-0.15%,0.02%,0.05%,0.25%,-0.03%,-0.14%,-0.07%,0.09%
8,-0.21%,-0.14%,0.00%,0.26%,0.23%,-0.08%,0.06%,-0.07%,0.03%,0.01%,0.08%,0.06%
9,-0.11%,-0.19%,-0.06%,0.23%,-0.16%,-0.25%,0.19%,-0.05%,-0.19%,-0.25%,-0.04%,-0.02%
10,0.13%,-0.01%,0.16%,0.21%,-0.05%,0.04%,-0.08%,-0.13%,-0.09%,0.04%,0.14%,0.04%


In [11]:
for col in pct_days.columns:
    pct_days.loc[~pct_days[col].isnull(), col] = pct_days.loc[~pct_days[col].isnull(), col].str.replace("%", "").astype(float)
pct_days

,Tables January,Tables February,Tables March,Tables April,Tables May,Tables June,Tables July,Tables August,Tables September,Tables October,Tables November,Tables December
1,NaN,0.16,0.3,0.28,0.19,0.18,0.26,-0.05,0.1,0.23,0.17,0.06
2,0.31,0.16,0.15,0.08,0.16,0.16,0.05,0.08,0.25,0.16,0.31,0.08
3,0.18,0.14,0.04,0.0,0.07,0.01,0.3,0.08,0.02,-0.04,0.39,-0.15
4,0.09,-0.07,0.2,-0.05,0.04,-0.01,NaN,-0.24,-0.15,0.17,0.21,0.14
5,0.09,-0.14,0.07,0.25,0.14,0.17,0.14,-0.14,-0.07,0.26,0.29,0.16
6,0.13,0.13,-0.05,0.18,-0.14,0.22,0.09,0.01,0.11,0.15,-0.08,0.16
7,-0.07,0.03,-0.25,-0.19,-0.15,0.02,0.05,0.25,-0.03,-0.14,-0.07,0.09
8,-0.21,-0.14,0.0,0.26,0.23,-0.08,0.06,-0.07,0.03,0.01,0.08,0.06
9,-0.11,-0.19,-0.06,0.23,-0.16,-0.25,0.19,-0.05,-0.19,-0.25,-0.04,-0.02
10,0.13,-0.01,0.16,0.21,-0.05,0.04,-0.08,-0.13,-0.09,0.04,0.14,0.04


In [12]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
# pct_days.iloc[:, :] = min_max_scaler.fit_transform(pct_days)
pct_days

,Tables January,Tables February,Tables March,Tables April,Tables May,Tables June,Tables July,Tables August,Tables September,Tables October,Tables November,Tables December
1,NaN,0.16,0.3,0.28,0.19,0.18,0.26,-0.05,0.1,0.23,0.17,0.06
2,0.31,0.16,0.15,0.08,0.16,0.16,0.05,0.08,0.25,0.16,0.31,0.08
3,0.18,0.14,0.04,0.0,0.07,0.01,0.3,0.08,0.02,-0.04,0.39,-0.15
4,0.09,-0.07,0.2,-0.05,0.04,-0.01,NaN,-0.24,-0.15,0.17,0.21,0.14
5,0.09,-0.14,0.07,0.25,0.14,0.17,0.14,-0.14,-0.07,0.26,0.29,0.16
6,0.13,0.13,-0.05,0.18,-0.14,0.22,0.09,0.01,0.11,0.15,-0.08,0.16
7,-0.07,0.03,-0.25,-0.19,-0.15,0.02,0.05,0.25,-0.03,-0.14,-0.07,0.09
8,-0.21,-0.14,0.0,0.26,0.23,-0.08,0.06,-0.07,0.03,0.01,0.08,0.06
9,-0.11,-0.19,-0.06,0.23,-0.16,-0.25,0.19,-0.05,-0.19,-0.25,-0.04,-0.02
10,0.13,-0.01,0.16,0.21,-0.05,0.04,-0.08,-0.13,-0.09,0.04,0.14,0.04


In [13]:
dates = last_day.select('date').row(0)

In [14]:
expiration_dates = np.unique(tickers.select('expiration_date').to_numpy().flatten()).tolist()
expiration_dates[::-1]

['2024-06-14',
 '2024-06-13',
 '2024-06-12',
 '2024-06-11',
 '2024-06-10',
 '2024-06-07',
 '2024-06-06',
 '2024-06-05',
 '2024-06-04',
 '2024-06-03',
 '2024-05-31',
 '2024-05-30',
 '2024-05-29',
 '2024-05-28',
 '2024-05-24',
 '2024-05-23',
 '2024-05-22',
 '2024-05-21',
 '2024-05-20',
 '2024-05-17',
 '2024-05-16',
 '2024-05-15',
 '2024-05-14',
 '2024-05-13',
 '2024-05-10',
 '2024-05-09',
 '2024-05-08',
 '2024-05-07',
 '2024-05-06',
 '2024-05-03',
 '2024-05-02',
 '2024-05-01',
 '2024-04-30',
 '2024-04-29',
 '2024-04-26',
 '2024-04-25',
 '2024-04-24',
 '2024-04-23',
 '2024-04-22',
 '2024-04-19',
 '2024-04-18',
 '2024-04-17',
 '2024-04-16',
 '2024-04-15',
 '2024-04-12',
 '2024-04-11',
 '2024-04-10',
 '2024-04-09',
 '2024-04-08',
 '2024-04-05',
 '2024-04-04',
 '2024-04-03',
 '2024-04-02',
 '2024-04-01',
 '2024-03-28',
 '2024-03-27',
 '2024-03-26',
 '2024-03-25',
 '2024-03-22',
 '2024-03-21',
 '2024-03-20',
 '2024-03-19',
 '2024-03-18',
 '2024-03-15',
 '2024-03-14',
 '2024-03-13',
 '2024-03-

In [111]:
starting_balance = 1000
buy_max = 100

sell_dates = dict()

balance = 100000

for i in range(len(last_day) - 3):
    strike_price = last_day.select('open').row(i)[0]
    timestamp = last_day.select('date').row(i)[0]
    
    date_str = timestamp.strftime('%Y-%m-%d')
    if date_str in sell_dates:
        balance += sell_dates[date_str]
        del sell_dates[date_str]
        
    if timestamp.strftime('%Y-%m-%d') not in expiration_dates:
        continue
    
    index = expiration_dates.index(timestamp.strftime('%Y-%m-%d'))
    
    timestamp_offset = expiration_dates[index + 3]
    
    
    day = timestamp.date().day - 1
    month = timestamp.date().month - 1
    action = pct_days.iloc[day, month]
    
    expiration_tickers = tickers.filter(pl.col("expiration_date") == timestamp_offset).filter(pl.col('strike_price') == np.round(strike_price)).unique(subset=['contract_type']).to_pandas()
    
    
    if action > 0:
        strike_ticker = expiration_tickers[expiration_tickers.contract_type == 'call']['ticker'].item()
    elif action < -0:
        strike_ticker = expiration_tickers[expiration_tickers.contract_type == 'put']['ticker'].item()
    else:
        continue
        
    h = history[strike_ticker]
    
    h1 = (
        h
        .group_by_dynamic("date", every='10m')
        .agg(
            pl.col('open').first(),
            pl.col('close').last(),
            pl.col("low").min(),
            pl.col("high").max()
        )
        .with_columns(pl.col('date').cast(pl.Datetime).dt.replace_time_zone("UTC"))
        .with_columns(pl.col('date').dt.convert_time_zone("America/New_York"))
        .with_columns(pl.col('date') + dt.timedelta(minutes=10))
        # .filter(pl.col('date').dt.date() == pl.lit(dt.datetime(2024, 5, 31)))
        # .filter(pl.col('date').dt.hour() < 16)
        # .filter(pl.col('date').dt.hour() >= 9)
        # .with_columns(pl.col('date').shift(-1).alias('date_shift'))
        # .filter(pl.col('date_shift').dt.month() != pl.col('date').dt.month())
        .filter(pl.col('date').dt.date() >= timestamp.date())
    )
    
    h2 = (
        h1
        .filter(pl.col('date').dt.date() == timestamp.date())
        .select(pl.col('open'))
    )
    
    if len(h2) == 0:
        continue
    
    h2 = np.min(h2.to_numpy().flatten()[0])
    
    num_shares = (balance * 0.2) // h2
    balance -= num_shares * h2
    
    # prices = h1.with_columns(pl.mean_horizontal(pl.col('open'), pl.col('close')).alias('row_mean')).select('row_mean').to_numpy().flatten()
    prices = h1.select("close").to_numpy().flatten()
    low_prices = h1.select("low").to_numpy().flatten()
    
    if len(prices) == 0:
        continue
        
        
    idx = prices >= h2 * 1.1
    
    idx2 = np.zeros(len(idx), dtype=bool)
    idx2[1:] = prices[:-1] > prices[1:]
    
    # if not np.any(idx & idx2):
    #     # idx = np.argmax(prices < h2 / 2)
    #     sell_price = low_prices[-1]
    # else:
    #     sell_price = low_prices[idx & idx2][0]
    if np.sum(idx) > 0:
        sell_price = prices[idx][0]
    else:
        l_index = np.argmax(low_prices < h2 / 2)
        if np.sum(l_index) == 0:
            l_index = -1
        sell_price = low_prices[l_index]
    
    sell_dates[timestamp_offset] = num_shares * sell_price
    
    
    greater = np.sum(idx)
    lesser = len(prices) - greater
    
    print(greater, lesser, action, timestamp.date(), balance, balance + sum(list(sell_dates.values())), h2, sell_price)

12 144 0.18 2023-01-03 80005.42 102075.22 6.07 6.7
59 95 0.09 2023-01-04 64004.38 105910.18 3.63 4.5
95 45 0.09 2023-01-05 51206.1 108142.73999999999 4.93 5.79
148 7 0.13 2023-01-06 58621.259999999995 110807.21999999999 1.54 1.82
23 133 -0.11 2023-01-09 62766.759999999995 112734.91 3.5 3.93
143 13 0.13 2023-01-10 62238.369999999995 115866.63999999998 3.13 3.76
38 118 0.08 2023-01-11 63648.84999999999 118028.13999999998 3.68 4.18
12 144 -0.09 2023-01-12 65015.51999999999 121201.25999999998 3.79 4.53
36 112 -0.09 2023-01-13 66967.67999999998 123463.25999999998 5.55 6.3
3 86 0.07 2023-01-17 68033.22999999998 126507.55999999998 3.91 4.61
21 130 0.02 2023-01-18 69970.50999999998 130515.07999999999 7.68 9.44
9 136 -0.05 2023-01-19 71178.14999999998 132652.56999999998 3.08 3.45
0 143 -0.26 2023-01-20 72987.77999999998 123484.55999999998 7.54 3.75
122 34 0.15 2023-01-23 75587.49999999997 126676.30999999997 1.48 1.73
21 135 -0.06 2023-01-24 76417.26999999996 128676.73999999996 2.96 3.27
129 27 